# LangChain Expression Language (LCEL) 🔗 

## Introduction
This notebook demonstrates the `use of the LangChain Expression Language (LCEL) to create and run various chains using LangChain's capabilities`. We will cover the setup, simple chains, more complex chains, and using OpenAI functions. Additionally, we will discuss handling fallbacks and improving output readability.

### Setup the Environment, OpenAI API Key  and Imports
First, we need to import the necessary libraries and set up the environment and the OpenAI API key:

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# Load environment variables from a .env file
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

# Print OpenAI API key (masked)
print(f"OPENAI_API_KEY: {os.getenv('OPENAI_API_KEY')[:5]}*****")

OPENAI_API_KEY: sk-ft*****


**Note** Ensure you have the required packages installed:

In [2]:
#!pip install pydantic==1.10.8

In [3]:
# Import the necessary modules from langchain
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [4]:
#Using the rich library to improve the readability of nested dictionary outputs.
%pip install rich

In [5]:
# Import the necessary modules from langchain
from rich import print
from rich.pretty import Pretty

## Simple Chain

We will start with a simple chain that generates a joke based on a given topic.

In [6]:
# Define the prompt template
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)

# Initialize the model and output parser
model = ChatOpenAI()
output_parser = StrOutputParser()

C:\Users\Michela\AppData\Local\Temp\ipykernel_15520\551848145.py:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI()


In [7]:
# Create the chain
chain = prompt | model | output_parser

In [8]:
# Invoke the chain with a topic
chain.invoke({"topic": "bears"})

'Why do bears have hairy coats?\n\nFur protection!'

In [9]:
# Invoke the chain with a topic
chain.invoke({"topic": "British people"})

'Why did the British man bring a ladder to the bar? Because he heard the drinks were on the house!'

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.We will now create a more complex chain using a retriever to supply user-provided inputs to the prompt.

In [10]:
# Import necessary modules from Langchain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [11]:
# Create a vector store from texts
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

C:\Users\Michela\AppData\Local\Temp\ipykernel_15520\2361442535.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding=OpenAIEmbeddings()
c:\Users\Michela\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [12]:
# Retrieve relevant documents based on a query1
retriever.get_relevant_documents("where did harrison work?")

C:\Users\Michela\AppData\Local\Temp\ipykernel_15520\3641190296.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("where did harrison work?")


[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [13]:
# Retrieve relevant documents based on a query2
retriever.get_relevant_documents("what do bears like to eat")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='harrison worked at kensho')]

- Creating the prompt template for a more complex chain:

In [14]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [15]:
# Define the runnable map and chain
from langchain.schema.runnable import RunnableMap

In [16]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [17]:
# Invoke the complex chain
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [18]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [19]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

### Bind and Using OpenAI Functions

We will demonstrate `how to bind and use OpenAI functions within LangChain`.

- Define the OpenAI Functions

In [20]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [21]:
# Define the prompt and runnable
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [22]:
runnable = prompt | model

In [23]:
# Invoke the model with a query
runnable.invoke({"input": "what is the weather in Amsterdam"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"EHAM"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 64, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-ca7e7662-a08b-49c5-9fc4-d6f6be035570-0')

In [24]:
import json

response = runnable.invoke({"input": "what is the weather in Amsterdam"})

# Check if response is a dictionary (JSON-like)
if isinstance(response, dict):
    print(json.dumps(response, indent=4))  # Pretty-print JSON
else:
    print(response)  # Print directly if it's a string or another type


AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"EHAM"}', 'name': 'weather_search'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 17,
            'prompt_tokens': 64,
            'total_tokens': 81,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-f1e9af8e-7bdb-4790-9fe8-9cb95f379a9d-0'
)

**Explanation output** It looks like the output from runnable.invoke() is a structured format and response.

In [25]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

- Bind the function to the model

In [26]:
# Bind the functions to the model
model = model.bind(functions=functions)

In [27]:
runnable = prompt | model

In [28]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"New England Patriots"}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 99, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-bf3404e7-4794-4aea-9dc3-920d2c90f6c7-0')

In [29]:
# import json

response = runnable.invoke({"input": "what is the latest update on the New England Patriots?"})

# Check if response is a dictionary (JSON-like)
if isinstance(response, dict):
    print(json.dumps(response, indent=4))  # Pretty-print JSON
else:
    print(response)  # Print directly if it's a string or another type

AIMessage(
    content='',
    additional_kwargs={
        'function_call': {'arguments': '{"team_name":"New England Patriots"}', 'name': 'sports_search'}
    },
    response_metadata={
        'token_usage': {
            'completion_tokens': 19,
            'prompt_tokens': 102,
            'total_tokens': 121,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-91f4cc1a-0389-4b43-8a83-7f2e5f3044f6-0'
)

**Explanation output** It looks like the output from runnable.invoke() is a structured response, but it's printed in a raw, unformatted way. You can extract and print it nicely in a structured format.

### Handling Fallbacks

In [30]:
from langchain.llms import OpenAI
# import json

In [31]:
# Initialize the fallback model
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="gpt-3.5-turbo-instruct"
)
simple_chain = simple_model | json.loads

C:\Users\Michela\AppData\Local\Temp\ipykernel_15520\4056484050.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  simple_model = OpenAI(


In [32]:
# Define a challenge
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [33]:
# Invoke the fallback chain
simple_model.invoke(challenge)

'\n\n{\n    "title": "Autumn Leaves",\n    "author": "Emily Dickinson",\n    "first_line": "The leaves are falling, one by one"\n}\n\n{\n    "title": "The Ocean\'s Song",\n    "author": "Pablo Neruda",\n    "first_line": "I hear the ocean\'s song, a symphony of waves"\n}\n\n{\n    "title": "A Winter\'s Night",\n    "author": "Robert Frost",\n    "first_line": "The snow falls softly, covering the ground"\n}'

In [34]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [35]:
chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

In [36]:
print(Pretty(chain.invoke(challenge)))

{
    'poem1': {
        'title': 'The Night Sky',
        'author': 'Emily Dickinson',
        'firstLine': 'The night is starry and the stars are blue.'
    },
    'poem2': {
        'title': 'Autumn Leaves',
        'author': 'Robert Frost',
        'firstLine': "My sorrow, when she's here with me, thinks these dark days of autumn rain are beautiful as 
days can be."
    },
    'poem3': {
        'title': 'Hope is the Thing with Feathers',
        'author': 'Emily Dickinson',
        'firstLine': 'Hope is the thing with feathers that perches in the soul.'
    }
}

In [37]:
final_chain = simple_chain.with_fallbacks([chain])

In [38]:
final_chain.invoke(challenge)

{'poem1': {'title': 'The Night Sky',
  'author': 'Emily Dickinson',
  'firstLine': 'The night is starry and the stars are blue.'},
 'poem2': {'title': 'Autumn Leaves',
  'author': 'Robert Frost',
  'firstLine': "My sorrow, when she's here with me, thinks these dark days of autumn rain are beautiful as days can be."},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul.'}}

In [39]:
print(Pretty(final_chain.invoke(challenge)))

{
    'poem1': {
        'title': 'The Rose',
        'author': 'Emily Dickinson',
        'firstLine': 'A rose by any other name would smell as sweet'
    },
    'poem2': {
        'title': 'The Road Not Taken',
        'author': 'Robert Frost',
        'firstLine': 'Two roads diverged in a yellow wood'
    },
    'poem3': {
        'title': 'Hope is the Thing with Feathers',
        'author': 'Emily Dickinson',
        'firstLine': 'Hope is the thing with feathers that perches in the soul'
    }
}

## Interface

In this context, the term "interface" ilikely refers to the way users interact with the LangChain library to build and run various chains. This involves defining prompts, models, and chains, and then invoking these chains with specific inputs to get desired outputs.  
Here the interface is the `set of functions and classes provided by LangChain`, such as `ChatPromptTemplate, ChatOpenAI, StrOutputParser, and RunnableMap`. These components allow users to define and execute chains in a structured and modular way.These components work together to provide a flexible and powerful interface for building and executing chains, enabling users to define complex workflows in a modular and readable manner.

In [40]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [41]:
chain.invoke({"topic": "bears"})

'Why did the bear bring a flashlight to the party? \n\nBecause he heard the drinks were on the house!'

In [42]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why don't bears wear shoes? \nBecause they have bear feet!",
 'Why are frogs so happy?\nBecause they eat whatever bugs them!']

In [43]:
chain.invoke({"topic": "British people"})

"Why do British people always carry umbrellas?\n\nBecause they can't handle a little bit of drizzle without panicking!"

In [44]:
chain.batch([{"topic": "British people"}, {"topic": "American people"}])

['Why do British people never play hide and seek?\n\nBecause good luck hiding when you have such terrible teeth!',
 'Why did the American man bring a ladder to the bar? \n\nBecause he heard the drinks were on the house!']

In [45]:
for t in chain.stream({"topic": "British people"}):
    print(t)

Why

do

British

people

always

...

themselves

in

hot

water

!

In [46]:
for t in chain.stream({"topic": "bears"}):
    print(t)

Why

don

't

bears

like

...

can

't

catch

it

!

**Explanation output** a joke formatted in a humorous way with additional spaces between each word. The joke goes:  
Q: Why did the bear bring a ladder to the bar?  
A: Because he heard the drinks were on the house!  

The punchline "the drinks were on the house" is a play on words, meaning both that the drinks are free (a common idiom) and that the drinks are literally located on the roof, hence the need for a ladder.

`The format with extra spaces between each word` can have several advantages such as `emphasis and clarity`, ``visual impact`, `engagement` and `humor`.

In [47]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship anymore!"

In [48]:
response = await chain.ainvoke({"topic": "British people"})
response

'Why did the British man bring a ladder to the bar?\n\nBecause he heard the drinks were on the house!'

## Conclusion
This notebook demonstrated the use of LangChain Expression Language (LCEL) to create and run various chains. We covered setup, simple and complex chains, using OpenAI functions, improving readability with the rich library, and handling fallbacks. These examples illustrate the flexibility and power of LangChain for building and running chains of different complexities.

Here's a short description of the interface as used in the notebook:  

Interface in LangChain Notebook
The LangChain notebook demonstrates how to use the LangChain interface to create and execute chains of operations. This interface includes:  
- Prompt Templates: ChatPromptTemplate allows users to define prompts with placeholders for dynamic content.
- Models: ChatOpenAI represents the language model used to process prompts and generate responses.
- Output Parsers: StrOutputParser parses the model's output into a desired format.
= Runnable Maps: RunnableMap allows for the creation of more complex chains by mapping inputs to specific functions or operations.